In [16]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2024-01-01'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' ",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data

In [17]:
hard_womens=hist_data_dog('HArd','Womens',1,2,datefilter)
hard_mens=hist_data_dog('HArd','Mens',1,2,datefilter)
clay_womens=hist_data_dog('Clay','Womens',1,2,datefilter)
clay_mens=hist_data_dog('Clay','Mens',1,2,datefilter)
temp=pd.concat([clay_womens,clay_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Loss'] = (temp['Fav_Win'] == 0).astype(int)
temp['Fav_Loss_Count'] = temp.groupby('Fav')['Fav_Loss'].cumsum()
temp['Dog_Win'] = (temp['Fav_Win'] == 0).astype(int)
temp['Dog_Win_Count'] = temp.groupby('Dog')['Dog_Win'].cumsum()


In [ ]:
temp
date_filter=temp[temp['Date']>'2024-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>2]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>2]
#oddsfilter=oddsfilter[oddsfilter['Elo_Fav']==oddsfilter['Dog']]
#oddsfilter=oddsfilter[oddsfilter['Dog_Top100']>.5]
#oddsfilter=oddsfilter[oddsfilter['Fav_Top100']<.5]
#oddsfilter=oddsfilter[oddsfilter['dog_rank']<200]
#oddsfilter=oddsfilter[oddsfilter['fav_rank']<300]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Womens']
len(oddsfilter[oddsfilter['Fav_Loss']==1])/len(oddsfilter),len(oddsfilter)
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter

Total Profit: -10123.0 1076 0.3429368029739777


,Date,Winner_Odds,Winner,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Games,Dog,Dog_Games,...,Dog_Top100,fav_rank_high,dog_rank_high,Fav_Win,DogWin,Fav_Loss,Fav_Loss_Count,Dog_Win,Dog_Win_Count,Profit
495,2024-03-25,1.68,Katarzyna Kawa,1.68,2.11,Katarzyna Kawa,Katarzyna Kawa,10,Carole Monnet,11,...,0.2,112.0,162.0,True,False,0,0,0,0,-100.0
496,2024-03-25,1.13,Gabriela Ruse Elena,1.13,5.48,Gabriela Ruse Elena,Gabriela Ruse Elena,10,Berfu Cengiz,5,...,0.0,51.0,228.0,True,False,0,0,0,0,-100.0
497,2024-03-25,1.65,Leyre Romero Gormaz,1.65,2.15,Leyre Romero Gormaz,Miriam Bulgaru,6,Miriam Bulgaru,7,...,0.0,156.0,183.0,True,False,0,0,0,0,-100.0
513,2024-03-26,1.65,Alexia Todoni Anca,1.65,2.14,Alexia Todoni Anca,Alexia Todoni Anca,5,Katarina Zavatska,6,...,0.0,226.0,103.0,True,False,0,0,0,0,-100.0
515,2024-03-26,1.99,Moyuka Uchijima,1.78,1.99,Veronika Erjavec,Veronika Erjavec,6,Moyuka Uchijima,5,...,0.0,179.0,104.0,False,True,1,1,1,1,99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5305,2024-12-09,1.77,Maja Chwalinska,1.77,1.97,Maja Chwalinska,Leolia Jeanjean,6,Leolia Jeanjean,11,...,0.4,128.0,102.0,True,False,0,0,0,4,-100.0
5306,2024-12-09,1.21,Maja Chwalinska,1.21,4.16,Maja Chwalinska,Maja Chwalinska,6,Ylena In Albon,11,...,0.4,128.0,110.0,True,False,0,0,0,2,-100.0
5307,2024-12-09,4.86,Ylena In Albon,1.17,4.86,Lourdes Carle Maria,Lourdes Carle Maria,11,Ylena In Albon,10,...,0.4,71.0,110.0,False,True,1,9,1,3,386.0
5715,2025-03-16,2.06,Irina Bara,1.72,2.06,Ipek Oz,Ipek Oz,10,Irina Bara,10,...,0.0,163.0,104.0,False,True,1,2,1,2,106.0


In [19]:
# Load the dataset
df = temp[(temp['Sex']!='Wobmens')&(temp['Fav_Odds']<2)&(temp['Fav_Odds']>1.4)]


# Feature selection and preprocessing
features = ['fav_percent', 'dog_percent', 'Elo_Fav_Elo', 'Elo_Dog_Elo', 
            'fav_rank', 'dog_rank', 'dog_rank_high', 'fav_rank_high','Fav_Top100','Dog_Top100','Dog_Win_Count','Fav_Loss_Count']

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Drop the 'Winner_Odds' column
df = df.drop(columns=['Winner_Odds'])


target = 'Fav_Win'

X = df[features]  # Features for training
y = df[target]  # Target variable

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42,shuffle=False)

# Model training
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Prediction and evaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Add predictions and actuals to the test set DataFrame
test_with_predictions = X_test.copy()
test_with_predictions['Actual'] = y_test
test_with_predictions['Predicted'] = y_pred

# Add Fav_Odds and Dog_Odds back to the test set for profit calculation
test_with_predictions = test_with_predictions.merge(
    df[['Date','Fav','Dog','Fav_Odds', 'Dog_Odds']], left_index=True, right_index=True
)

# Define the stake amount
stake = 100  # Example stake amount

# Profit calculation
def calculate_profit(row):
    if row['Predicted'] == 1:  # If the model predicts the favorite wins
        if row['Actual'] == 1:  # If the prediction is correct
            return stake * (row['Fav_Odds'] - 1)  # Profit based on favorite odds
        else:
            return -stake  # Loss of the stake
    else:  # If the model predicts the underdog wins
        if row['Actual'] == 0:  # If the prediction is correct
            return stake * (row['Dog_Odds'] - 1)  # Profit based on underdog odds
        else:
            return -stake  # Loss of the stake

test_with_predictions['Profit'] = test_with_predictions.apply(calculate_profit, axis=1)

# Save the output DataFrame to clipboard
fav_predictions=test_with_predictions[test_with_predictions['Predicted']==1].copy()
# Calculate overall profit
fav_profit = fav_predictions['Profit'].sum()
print(f"Fav Profit: ${fav_profit:.2f}",len(fav_predictions))

dog_predictions=test_with_predictions[test_with_predictions['Predicted']==0].copy()
# Calculate overall profit
dog_profit = dog_predictions['Profit'].sum()
print(f"Dog Profit: ${dog_profit:.2f}", len(dog_predictions))
dog_predictions.to_clipboard()

Accuracy: 0.5608465608465608
              precision    recall  f1-score   support

       False       0.49      0.60      0.54        82
        True       0.63      0.53      0.58       107

    accuracy                           0.56       189
   macro avg       0.56      0.57      0.56       189
weighted avg       0.57      0.56      0.56       189

Fav Profit: $15.00 90
Dog Profit: $1275.00 99


In [21]:
import numpy as np

# Define the odds bands (10c increments from 1.00 to 2.00)
bins = np.arange(1.0, 2.1, 0.1)  # Create bins: [1.0, 1.1, ..., 2.0]
labels = [f"{bins[i]:.1f} to {bins[i+1]:.1f}" for i in range(len(bins) - 1)]

# Create a new column for odds bands
dog_predictions['Fav_Odds_Band'] = pd.cut(dog_predictions['Fav_Odds'], bins=bins, labels=labels, right=False)

# Add a column to indicate if the prediction was correct
dog_predictions['Correct_Prediction'] = dog_predictions['Actual'] == dog_predictions['Predicted']

# Group by Fav_Odds_Band and calculate Profit, count of records, and winning percentage
grouped_dog = dog_predictions.groupby('Fav_Odds_Band').agg(
    Total_Profit=('Profit', 'sum'),
    Record_Count=('Profit', 'count'),
    Winning_Percentage=('Correct_Prediction', lambda x: (x.sum() / len(x) * 100) if len(x) > 0 else 0)
).reset_index()

# Print the results
print(grouped_dog)



import numpy as np

# Define the odds bands (10c increments from 1.00 to 2.00)
bins = np.arange(1.0, 2.1, 0.1)  # Create bins: [1.0, 1.1, ..., 2.0]
labels = [f"{bins[i]:.1f} to {bins[i+1]:.1f}" for i in range(len(bins) - 1)]

# Create a new column for odds bands
fav_predictions['Fav_Odds_Band'] = pd.cut(fav_predictions['Fav_Odds'], bins=bins, labels=labels, right=False)

# Add a column to indicate if the prediction was correct
fav_predictions['Correct_Prediction'] = fav_predictions['Actual'] == fav_predictions['Predicted']

# Group by Fav_Odds_Band and calculate Profit, count of records, and winning percentage
grouped_fav = fav_predictions.groupby('Fav_Odds_Band').agg(
    Total_Profit=('Profit', 'sum'),
    Record_Count=('Profit', 'count'),
    Winning_Percentage=('Correct_Prediction', lambda x: (x.sum() / len(x) * 100) if len(x) > 0 else 0)
).reset_index()

# Print the results
print(grouped_fav)

  Fav_Odds_Band  Total_Profit  Record_Count  Winning_Percentage
0    1.0 to 1.1           0.0             0            0.000000
1    1.1 to 1.2           0.0             0            0.000000
2    1.2 to 1.3           0.0             0            0.000000
3    1.3 to 1.4           0.0             0            0.000000
4    1.4 to 1.5        1315.0            24           58.333333
5    1.5 to 1.6         375.0            15           53.333333
6    1.6 to 1.7         122.0            25           48.000000
7    1.7 to 1.8        -286.0            21           42.857143
8    1.8 to 1.9        -251.0            14           42.857143
9    1.9 to 2.0           0.0             0            0.000000
  Fav_Odds_Band  Total_Profit  Record_Count  Winning_Percentage
0    1.0 to 1.1  0.000000e+00             0            0.000000
1    1.1 to 1.2  0.000000e+00             0            0.000000
2    1.2 to 1.3  0.000000e+00             0            0.000000
3    1.3 to 1.4  0.000000e+00           

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1508\2049959661.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_dog = dog_predictions.groupby('Fav_Odds_Band').agg(
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1508\2049959661.py:38: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_fav = fav_predictions.groupby('Fav_Odds_Band').agg(


In [22]:
# Current date and time
current_datetime = datetime.datetime.now()

# Convert to string
date_string = current_datetime.strftime("%Y-%m-%d")

today=pd.read_pickle('.\Clay_Today')
today=today[today['Fav_Odds']>1.4]
today_features=today[features]
today['Predicted']=model.predict(today_features)
probabilities = model.predict_proba(today_features)

today['Predicted_Prob'] = probabilities[:, 0]  # Column 1 corresponds to the positive class
today['Date']=date_string
dog=today[today['Predicted']==False][['Date','Time','Dog','Predicted','Predicted_Prob','Dog_Odds','Fav_Odds','Fav']]
dog = dog[dog['Dog_Odds'] >1]
dog = dog[dog['Predicted_Prob']>0.45]
dog

<>:7: SyntaxWarning: invalid escape sequence '\C'
<>:7: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1508\3906015690.py:7: SyntaxWarning: invalid escape sequence '\C'
  today=pd.read_pickle('.\Clay_Today')


,Date,Time,Dog,Predicted,Predicted_Prob,Dog_Odds,Fav_Odds,Fav
0,2025-03-17,00:00,Orlando Luz,False,0.60,2.32,1.56,Gabriele Pennaforti
1,2025-03-17,00:05,Martin Klizan,False,0.64,2.03,1.70,Mirza Basic
3,2025-03-17,00:55,Abedallah Shelbayh,False,0.57,2.24,1.59,Raul Brancaccio
5,2025-03-17,05:10,Daniel Elahi Galan,False,0.60,2.08,1.69,Thiago Monteiro
8,2025-03-17,18:35,Noma Noha Akugue,False,0.61,2.74,1.41,Dalma Galfi
10,2025-03-17,20:00,David Jorda Sanchis,False,0.54,2.40,1.52,Daniel Merida Aguilar
11,2025-03-17,20:30,Lilian Marmousez,False,0.51,2.41,1.51,Marko Topo
12,2025-03-17,20:30,Michael Vrbensky,False,0.66,2.08,1.68,Mirza Basic
17,2025-03-17,22:00,Elias Ymer,False,0.56,2.59,1.46,Dino Prizmic
19,2025-03-17,23:00,Sebastian Ofner,False,0.53,2.23,1.60,Kimmer Coppejans


In [23]:
date_filter=pd.read_pickle('.\Clay_Today')
oddsfilter=date_filter[date_filter['Fav_Odds']>1].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>2]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>2]
#oddsfilter=oddsfilter[oddsfilter['Elo_Fav']==oddsfilter['Dog']]
#oddsfilter=oddsfilter[oddsfilter['Dog_Top100']>.5]
#oddsfilter=oddsfilter[oddsfilter['Fav_Top100']<.5]
#oddsfilter=oddsfilter[oddsfilter['dog_rank']<200]
#oddsfilter=oddsfilter[oddsfilter['fav_rank']<300]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Womens']
oddsfilter

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1508\2569379612.py:1: SyntaxWarning: invalid escape sequence '\C'
  date_filter=pd.read_pickle('.\Clay_Today')


,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,...,Fav_Top100,Dog_Top100,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count
6,18:30,1.40,2.78,Tamara Zidansek,Tamara Zidansek,100%,2,Irina Bara,25%,8,...,0.3,0.0,0.60,0.33,0.53,0.36,0.6,0.2,4.0,3.0
8,18:35,1.41,2.74,Dalma Galfi,Noma Noha Akugue,67%,3,Noma Noha Akugue,60%,10,...,0.2,0.2,0.67,0.32,0.00,0.00,0.4,0.2,4.0,9.0


In [24]:
fav=today[today['Predicted']==True][['Date','Time','Fav','Predicted','Predicted_Prob','Dog_Odds','Fav_Odds','Dog']]
fav = fav[fav['Dog_Odds'] >1]
fav = fav[fav['Predicted_Prob']<0.51]
fav

,Date,Time,Fav,Predicted,Predicted_Prob,Dog_Odds,Fav_Odds,Dog
7,2025-03-17,18:30,Elena Pridankina,True,0.03,2.41,1.52,Mona Barthel
9,2025-03-17,19:00,Andrej Martin,True,0.47,1.95,1.76,Franco Agamenone
13,2025-03-17,20:30,Daniel Michalski,True,0.47,2.02,1.71,Zdenek Kolar
14,2025-03-17,21:30,Nicolas Alvarez Varona,True,0.47,2.43,1.51,Gabriele Pennaforti
16,2025-03-17,22:00,Hans Rehberg Max,True,0.49,2.60,1.45,Enrico Dalla Valle
20,2025-03-17,23:00,Nicolai Budkov Kjaer,True,0.41,2.53,1.48,Javier Barranco Cosano
23,2025-03-17,23:30,Jozef Kovalik,True,0.37,2.46,1.50,Matej Dodig


In [25]:
fav.to_sql('DailyPredictions_Fav_Clay',con=devengine,index=False,if_exists='append')
dog.to_sql('DailyPredictions_Dog_Clay',con=devengine,index=False,if_exists='append')
predictions=pd.read_sql_table('DailyPredictions_Dog_Clay',con=devengine)
results=temp
merged_df = pd.merge(
    predictions, 
    results[['Date', 'Fav', 'Dog', 'Winner']],  # Select only the Winner column from results
    on=['Date', 'Fav', 'Dog'], 
    how='left'  # Use left join to retain all rows from predictions
)
merged_df_filtered=merged_df[merged_df['Winner'].notna()].copy()
merged_df_filtered=merged_df_filtered[merged_df_filtered['Predicted_Prob']<0.6]
#merged_df_filtered=merged_df_filtered[merged_df_filtered['Predicted_Prob']>0.6]

merged_df_filtered['Profit'] = merged_df_filtered.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = merged_df_filtered['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(merged_df_filtered),len(merged_df_filtered[merged_df_filtered['Dog']==merged_df_filtered['Winner']])/len(merged_df_filtered))
merged_df_filtered


Total Profit: -104.99999999999997 29 0.41379310344827586


,Date,Time,Dog,Predicted,Predicted_Prob,Dog_Odds,Fav_Odds,Fav,Winner,Profit
0,2025-02-16,04:05,Federico Coria,False,0.56,1.90,1.86,Roman Andres Burruchaga,Roman Andres Burruchaga,-100.0
2,2025-02-16,04:10,Tomas Barrios Vera,False,0.52,2.27,1.61,Cristian Garin,Tomas Barrios Vera,127.0
3,2025-02-16,04:10,Laslo Djere,False,0.55,2.25,1.64,Joao Fonseca,Joao Fonseca,-100.0
8,2025-02-18,05:40,Alexander Shevchenko,False,0.54,2.10,1.73,Felipe Meligeni Alves,Alexander Shevchenko,110.0
10,2025-02-18,11:10,Thiago Seyboth Wild,False,0.58,2.14,1.70,Jaume Munar,Jaume Munar,-100.0
12,2025-02-19,09:25,Camilo Ugo Carabelli,False,0.59,2.34,1.59,Pedro Martinez,Camilo Ugo Carabelli,134.0
14,2025-02-19,09:25,Camilo Ugo Carabelli,False,0.59,2.34,1.59,Pedro Martinez,Camilo Ugo Carabelli,134.0
18,2025-02-22,11:05,Alexandre Muller,False,0.50,2.68,1.47,Francisco Cerundolo,Alexandre Muller,168.0
20,2025-02-25,09:10,Juan Pablo Ficovich,False,0.52,2.52,1.51,Cristian Garin,Cristian Garin,-100.0
21,2025-02-25,19:20,Joel Schwaerzler,False,0.51,1.99,1.75,Zdenek Kolar,Joel Schwaerzler,99.0


In [26]:
predictions=pd.read_sql_table('DailyPredictions_Fav_Clay',con=devengine)
results=temp
merged_df = pd.merge(
    predictions, 
    results[['Date', 'Fav', 'Dog', 'Winner']],  # Select only the Winner column from results
    on=['Date', 'Fav', 'Dog'], 
    how='left'  # Use left join to retain all rows from predictions
)
merged_df_filtered=merged_df[merged_df['Winner'].notna()].copy()
merged_df_filtered=merged_df_filtered[merged_df_filtered['Predicted_Prob']>0.1]
merged_df_filtered['Profit'] = merged_df_filtered.apply(
    lambda row: (row['Dog_Odds'] * stake)-stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = merged_df_filtered['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(merged_df_filtered),len(merged_df_filtered[merged_df_filtered['Dog']==merged_df_filtered['Winner']])/len(merged_df_filtered))
merged_df_filtered

Total Profit: -34.0 46 0.43478260869565216


,Date,Time,Fav,Predicted,Predicted_Prob,Dog_Odds,Fav_Odds,Dog,Winner,Profit
0,2025-02-16,05:55,Yannick Hanfmann,True,0.43,2.25,1.61,Marco Trungelliti,Marco Trungelliti,125.0
1,2025-02-17,05:05,Francisco Cerundolo,True,0.48,2.28,1.63,Joao Fonseca,Joao Fonseca,128.0
2,2025-02-18,06:40,Nicolas Jarry,True,0.44,2.76,1.43,Juan Manuel Cerundolo,Nicolas Jarry,-100.0
3,2025-02-18,07:50,Francisco Comesana,True,0.39,2.15,1.69,Gustavo Heide,Francisco Comesana,-100.0
4,2025-02-19,05:40,Dusan Lajovic,True,0.47,2.73,1.45,Damir Dzumhur,Damir Dzumhur,173.0
5,2025-02-19,07:25,Mariano Navone,True,0.37,2.45,1.54,Roberto Carballes Baena,Mariano Navone,-100.0
6,2025-02-19,05:40,Dusan Lajovic,True,0.47,2.73,1.45,Damir Dzumhur,Damir Dzumhur,173.0
7,2025-02-19,07:25,Mariano Navone,True,0.37,2.45,1.54,Roberto Carballes Baena,Mariano Navone,-100.0
8,2025-02-25,09:05,Thiago Seyboth Wild,True,0.46,2.42,1.55,Gustavo Heide,Gustavo Heide,142.0
11,2025-02-25,18:10,Alex Marti Pujolras,True,0.45,2.45,1.51,Dominik Kellovsky,Alex Marti Pujolras,-100.0
